In [61]:
import pandas as pd
import numpy as np

In [62]:
def calculateAccuracy(predictionTable):
    accuracy = (predictionTable[0,0] + predictionTable[1,1]) / (predictionTable[0,0] + predictionTable[0,1] + predictionTable[1,0] + predictionTable[1,1])
#     print(accuracy)
    return accuracy

In [63]:
def calculatePrecision(predictionTable):
    precision = (predictionTable[0,0]) / (predictionTable[0,0] +  predictionTable[1,0])
    if (predictionTable[0,0] +  predictionTable[1,0]) == 0 :
        precision = 0
    
    #print(precision)
    return precision

In [64]:
def calculateRecall(predictionTable):
    recall = (predictionTable[0,0]) / (predictionTable[0,0] +  predictionTable[0,1])
    if (predictionTable[0,0] +  predictionTable[0,1]) == 0 :
        recall = 0
    #print(recall)
    return recall 

In [65]:
def calculateFmeasure(predictionTable):
    #2*(Recall * Precision) / (Recall + Precision)
    recall = calculateRecall(predictionTable)
    precision = calculatePrecision(predictionTable)
    
    f_measure = (2 * recall * precision) / (recall + precision)
    if recall == precision == 0 :
        f_measure = 0
        
#     print(precision)
#     print(recall)
#     print(f_measure)
    return f_measure

In [66]:
def getKFoldData(path, kfold ):
    train_arr = list()
    test_arr = list()
    data = pd.read_csv(path)
    dataLen = len(data)
    segmentSize = int(dataLen/kfold)
#     np.random.seed(kfold)
#     seedList = np.random.randint(low=0, high=200, size=kfold)

#     for seed in seedList:
#         tempDf = data.copy()
#         np.random.seed(seed)
#         tempDf.apply(np.random.shuffle, axis=0)
#         #print(tempDf.iloc[0:int(dataLen/10)])
#         test_arr.append(tempDf.iloc[0:segmentSize])
#         train_arr.append(tempDf.iloc[segmentSize: ].reset_index())
    tempDf = data.copy()
    np.random.seed(kfold)
    tempDf.apply(np.random.shuffle, axis=0)
    
    
    for i in range(0, kfold):
            
            #print(tempDf.iloc[0:int(dataLen/10)])
            testSegment = tempDf.iloc[(i*segmentSize): (i*segmentSize + segmentSize)]
            test_arr.append(testSegment)
            train_arr.append(tempDf.drop(tempDf.index[(i*segmentSize): (i*segmentSize + segmentSize)]))
            #train_arr.append(tempDf.iloc[int(dataLen/10): ].reset_index())

    return train_arr, test_arr

In [69]:
def spam_ham_detect():

    classIndex = 57
    filePath = 'spambase.csv'
    kFold = 10
    accuracy = 0
    precision = 0.0
    recall = 0.0
    f_measure = 0.0

    data = pd.read_csv(filePath, header = None)
    rowCount = data.shape[1]
    hamProb = np.zeros((rowCount-1))
    spamProb = np.zeros((rowCount-1))

    train_arr, test_arr = getKFoldData(filePath, kFold)


    for it in range(0, kFold):
        
        predictionTable = np.zeros((2,2))
        
        hamData = train_arr[it][train_arr[it].iloc[:, classIndex] == 0]
        spamData = train_arr[it][train_arr[it].iloc[:, classIndex] == 1]

        for i in range(0, rowCount-1):
            dataSum = train_arr[it].iloc[:, i].sum()
            hamProb[i] = hamData.iloc[:, i].sum()
            spamProb[i] = spamData.iloc[:, i].sum()
           
        
        for index,test in test_arr[it].iterrows():
            
            ham = 1.0
            spam = 1.0
            iterator = 0
            for value in test.iteritems():
               
                if value != 0:
                    if hamProb[iterator] == 0 or spamProb[iterator] == 0:
                        continue
                    ham *= hamProb[iterator]
                    spam *= spamProb[iterator]
            iterator += 1
            
            predictedValue = 0
            
            if spam > ham:
                predictedValue = 1
                
    
            if predictedValue == test.iloc[classIndex] == 1 :
                    predictionTable[0,0] += 1

            if predictedValue == 0 and test.iloc[classIndex] == 1 :
                predictionTable[0,1] += 1

            if predictedValue == 1 and test.iloc[classIndex] == 0 :
                predictionTable[1,0] += 1

            if predictedValue == 0 and test.iloc[classIndex] == 0 :
                predictionTable[1,1] += 1
            
            accuracy += calculateAccuracy(predictionTable)
#             precision += calculatePrecision(predictionTable)
#             recall += calculateRecall(predictionTable)
#             f_measure += calculateFmeasure(predictionTable)
    print("accuracy: ",accuracy/kFold)
#     print("precision: ", precision/kFold)
#     print("recall: ",recall/kFold)
#     print("f_measure: ", f_measure/kFold)

In [70]:
spam_ham_detect()

accuracy:  91.90995773215387
